# Circuit learning module: Lambeq manually with SPSA and JAX

This module performs the optimization of the parametrized circuit manually compared to Lambeq's automatic QuantumTrainer class. I created this because I wanted to have more control over the optimization process and debug it better. The code is based on the workflow presented in https://github.com/CQCL/Quanthoven.

In [1]:
import warnings
import json
import os
import sys
import glob
from math import ceil
from pathlib import Path
from jax import numpy as np
from sympy import default_sort_key
import numpy
import pickle
import matplotlib.pyplot as plt

import jax
from jax import jit
from noisyopt import minimizeSPSA, minimizeCompass

from discopy.quantum import Circuit
from discopy.tensor import Tensor
from discopy.utils import loads
#from pytket.extensions.qiskit import AerBackend
#from pytket.extensions.qulacs import QulacsBackend
#from pytket.extensions.cirq import CirqStateSampleBackend
backend = None

from utils import get_symbols, construct_data_and_labels, select_circuits, read_diagrams, create_labeled_classes, bin_class_loss, multi_class_loss, bin_class_acc, multi_class_acc, visualize_result_noisyopt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

warnings.filterwarnings('ignore')
this_folder = os.path.abspath(os.getcwd())
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
#os.environ["JAX_PLATFORMS"] = "cpu"

EPOCHS = 500
SEED = 0

# This avoids TracerArrayConversionError from jax
Tensor.np = np

rng = numpy.random.default_rng(SEED)
numpy.random.seed(SEED)

## Read circuit data

We read the circuits from the pickled files. Select if we perform binary classification or multi-class classification. Give number of qubits to create classes:
- 1 qubits -> 2^1 = 2 classes i.e. binary classification
- 2 qubits -> 2^2 = 4 classes
- ...
- 5 qubits -> 2^5 = 32 classes, etc.

In [2]:
# Select workload
workload = "execution_time"
#workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

loss = multi_class_loss
acc = multi_class_acc

if classification == 1:
    loss = bin_class_loss
    acc = bin_class_acc

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//" + workload + "//" + workload_size + "//circuits//" + str(classification) + "//" + str(layers) + "_layer//" + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count) + "_n_wire_count//"

training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [3]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [4]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification)
test_data_labels = create_labeled_classes(test_data, classification)
validation_data_labels = create_labeled_classes(validation_data, classification)

## Lambeq optimizer

## Model

In [5]:
def make_pred_fn(circuits):
    # In the case we want to use other backends. 
    # Currently does not work properly.
    if backend:
        compiled_circuits1 = backend.get_compiled_circuits([c.to_tk() for c in circuits])
        circuits = [Circuit.from_tk(c) for c in compiled_circuits1]
        
    circuit_fns = [c.lambdify(*parameters) for c in circuits]
    
    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns), backend = backend)
        res = []
        
        for output in outputs:
            predictions = np.abs(output.array) + 1e-9
            ratio = predictions / predictions.sum()
            res.append(ratio)
            
        return np.array(res)
    return predict

## Loss function and evaluation

In [6]:
def make_cost_fn(pred_fn, labels):
    def cost_fn(params, **kwargs):
        predictions = pred_fn(params)

        cost = loss(predictions, labels) #-np.sum(labels * np.log(predictions)) / len(labels)  # binary cross-entropy loss
        costs.append(cost)

        accuracy = acc(predictions, labels) #np.sum(np.round(predictions) == labels) / len(labels) / 2  # half due to double-counting
        accuracies.append(accuracy)

        return cost

    costs, accuracies = [], []
    return cost_fn, costs, accuracies

## Minimization with noisyopt

In [7]:
def initialize_parameters(old_params, old_values, new_params):
    new_values = list(numpy.array(rng.random(len(new_params))))
    old_param_dict = {}
    for p, v in zip(old_params, old_values):
        old_param_dict[p] = v
        
    parameters = sorted(set(old_params + new_params), key=default_sort_key)
    values = []
    for p in parameters:
        if p in old_param_dict:
            values.append(old_param_dict[p])
        else:
            values.append(new_values.pop())
            
    return parameters, np.array(values)

In [8]:
def grid_search():
    return None

In [ ]:
EPOCHS = 3000
syms = {}
all_training_keys = list(training_circuits.keys())
initial_circuit_keys = all_training_keys[:5]
current_training_circuits = {}

for k in initial_circuit_keys:
    current_training_circuits[k] = training_circuits[k]
    
syms = get_symbols(current_training_circuits)
parameters = sorted(syms, key=default_sort_key)
init_params_spsa = np.array(rng.random(len(parameters)))

run = 0
for i, key in enumerate(all_training_keys[5:]):
    print("Progress: ", round((i + 10)/len(all_training_keys), 3))
    
    # Skip cases when the number of parameters did not grow when new circuits were added
    if len(syms) == len(get_symbols(current_training_circuits)) and i > 0:
        if i != len(all_training_keys[10:]):
            current_training_circuits[key] = training_circuits[key]
            new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
            parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)
            continue
    else:
        run += 1
    
    # Select those circuits from test and validation circuits which share the parameters with the current training circuits
    current_validation_circuits = select_circuits(current_training_circuits, validation_circuits)
    current_test_circuits = select_circuits(current_training_circuits, test_circuits)
    
    if len(current_validation_circuits) == 0 or len(current_test_circuits) == 0:
        continue
    
    # Create lists with circuits and their corresponding label
    training_circuits_l, training_data_labels_l = construct_data_and_labels(current_training_circuits, training_data_labels)
    validation_circuits_l, validation_data_labels_l = construct_data_and_labels(current_validation_circuits, validation_data_labels)
    test_circuits_l, test_data_labels_l = construct_data_and_labels(current_test_circuits, test_data_labels)
    
    # Limit the number of validation and test circuits to 20% of number of the training circuits
    #val_test_circ_size = ceil(len(current_training_circuits))
    #if len(current_validation_circuits) > val_test_circ_size:
    #    validation_circuits_l = validation_circuits_l[:val_test_circ_size]
    #    validation_data_labels_l = validation_data_labels_l[:val_test_circ_size]
    #if len(current_test_circuits) > val_test_circ_size:
    #    test_circuits_l = test_circuits_l[:val_test_circ_size]
    #    test_data_labels_l = test_data_labels_l[:val_test_circ_size]
    
    print(f"Number of training circuits: {len(training_circuits_l)}   ",
          f"Number of validation circuits: {len(validation_circuits_l)}   ",
          f"Number of test circuits: {len(test_circuits_l)}   ",
          f"Number of parameters in model: {len(set([sym for circuit in training_circuits_l for sym in circuit.free_symbols]))}")
    
    train_pred_fn = jit(make_pred_fn(training_circuits_l))
    dev_pred_fn = jit(make_pred_fn(validation_circuits_l))
    test_pred_fn = make_pred_fn(test_circuits_l)
    
    train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
    dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)
    
    def callback_fn(xk):
        #print(xk)
        valid_loss = dev_cost_fn(xk)
        train_loss = numpy.around(min(float(train_costs[-1]), float(train_costs[-2])), 4)
        train_acc = numpy.around(min(float(train_accs[-1]), float(train_accs[-2])), 4)
        valid_acc = numpy.around(float(dev_accs[-1]), 4)
        iters = int(len(train_accs)/2)
        #if iters % 100 == 0:
        print(
                #f"Params = {xk}, ",
                f"Epoch: {iters}   ",
                f"train/loss: {train_loss}   ",
                f"valid/loss: {numpy.around(float(valid_loss), 4)}   ",
                f"train/acc: {train_acc}   ",
                f"valid/acc: {valid_acc}", file=sys.stderr
            )
        return valid_loss
    
    a_value = 0.07 #0.0053 #0.0053
    c_value = 0.01 #0.00017 #0.0185
    #a_values = [0.005, 0.0051, 0.0052, 0.0053, 0.0054, 0.0055]
    #c_values = [0.00017, 0.000175, 0.00018, 0.000185, 0.00019, 0.000195]
    #a_values = [0.045, 0.05, 0.07, 0.1, 0.15]
    #c_values = [0.0045, 0.005, 0.007, 0.01, 0.013, 0.015]
    #for i, a_value in enumerate(a_values):
    #    for c_value in c_values:
    
    #for a_value in a_values:
    #    for c_value in c_values:
            #print("a: ", a_value)
            #print("c: ", c_value)
            #train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, training_data_labels_l)
            #dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, validation_data_labels_l)
            
    #result = minimizeSPSA(train_cost_fn, x0=init_params_spsa, a = a_value, c = c_value, niter=EPOCHS, callback=callback_fn)
    result = minimizeCompass(train_cost_fn, x0=init_params_spsa, redfactor=2.0, deltainit=1.0, deltatol=0.001, feps=1e-15, errorcontrol=True, funcNinit=30, funcmultfactor=2.0, paired=True, alpha=0.05, callback=callback_fn)

    figure_path = this_folder + "//results//" + workload_size + "_noisyopt_compass_" + str(classification) + "_" + str(layers) + "_" + str(single_qubit_params) + "_" + str(run) + ".png"
    visualize_result_noisyopt(result, make_cost_fn, test_pred_fn, test_data_labels_l, train_costs, train_accs, dev_costs, dev_accs, figure_path)
    
    run += 1
    EPOCHS += 100
    syms = get_symbols(current_training_circuits)
    
    # Extend for the next optimization round
    current_training_circuits[key] = training_circuits[key]
    new_parameters = sorted(get_symbols({key: training_circuits[key]}), key=default_sort_key)
    parameters, init_params_spsa = initialize_parameters(parameters, result.x, new_parameters)

Progress:  0.022
Number of training circuits: 5    Number of validation circuits: 5    Number of test circuits: 8    Number of parameters in model: 103


Params = [0.6369617  0.26978672 0.04097353 ... 0.47998792 0.23237292 0.8018806 ],  Epoch: 3090    train/loss: 1.4855    valid/loss: 1.6128    train/acc: 0.6    valid/acc: 0.4
Params = [0.6369617  0.26978672 0.04097353 ... 0.47998792 0.23237292 0.8018806 ],  Epoch: 6045    train/loss: 1.4855    valid/loss: 1.6128    train/acc: 0.6    valid/acc: 0.4
Params = [ 1.1369617  -0.23021328  0.04097353 ...  0.47998792  0.23237292
  0.8018806 ],  Epoch: 9105    train/loss: 1.1664    valid/loss: 1.6737    train/acc: 0.8    valid/acc: 0.4
Params = [ 1.1369617  -0.23021328  0.04097353 ...  0.47998792  0.23237292
  0.8018806 ],  Epoch: 12090    train/loss: 1.1565    valid/loss: 1.6769    train/acc: 0.8    valid/acc: 0.4
Params = [ 1.1369617  -0.23021328  0.04097353 ...  0.47998792  0.23237292
  0.8018806 ],  Epoch: 14190    train/loss: 1.2926    valid/loss: 1.6769    train/acc: 0.6    valid/acc: 0.4
Params = [ 1.1369617  -0.48021328  0.04097353 ...  0.47998792 -0.01762708
  0.8018806 ],  Epoch: 17265

Test accuracy: 0.25
Progress:  0.025
Number of training circuits: 6    Number of validation circuits: 18    Number of test circuits: 19    Number of parameters in model: 130


Params = [2.1369617  0.50025547 0.04097353 ... 0.56047595 0.8097108  0.5540905 ],  Epoch: 3885    train/loss: 0.249    valid/loss: 7.6251    train/acc: 1.0    valid/acc: 0.5
Params = [2.1369617  0.50025547 0.04097353 ... 0.56047595 0.8097108  0.5540905 ],  Epoch: 7725    train/loss: 0.2489    valid/loss: 7.6253    train/acc: 1.0    valid/acc: 0.5
